## The contents include:

* 1. Querying the SRA database
* 2. Downloading data from the SRA database
* 3. Reading FASTQ files in R
* 4. Reading alignment data
* 5. Preprocessing the raw NGS data

## 1. Querying the SRA database

Most DNA sequencing data are stored [Sequence Read Archive (SRA) database](http://www.ncbi.nlm.nih.gov/sra) in terms of short reads generated by high-throughput sequencing obtained via different platforms. The sequences are usually less than 1,000 base pairs in length. At first, we need to know how to find sequences what we need. This part will introduce steps to query and access the data residing in the SRA database from within R. <br>

The following steps will help us run the desired queries on the SRA database.

In [1]:
if (!requireNamespace("BiocManager", quietly = TRUE))
install.packages("BiocManager")
BiocManager::install("SRAdb")

Bioconductor version 3.9 (BiocManager 1.30.10), R 3.6.2 (2019-12-12)

Installing package(s) 'SRAdb'



package 'SRAdb' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Master\AppData\Local\Temp\RtmpCEfOQw\downloaded_packages


Old packages: 'BH', 'bit', 'caTools', 'cli', 'FactoMineR', 'fansi', 'farver',
  'GetoptLong', 'ggridges', 'gplots', 'hms', 'knitr', 'leaps', 'multcomp',
  'mvtnorm', 'PMA', 'precrec', 'prettyunits', 'pROC', 'RCurl', 'RSQLite',
  'Rttf2pt1', 'shinyjs', 'stringi', 'tinytex', 'xfun', 'XML', 'xts', 'zoo'



In [13]:
library(SRAdb)
sqlFile <- getSRAdbFile() # this code helps to download the metadata file from sra server.
sraCon <- dbConnect(SQLite(),sqlFile)

Unzipping...




ERROR: Error: database disk image is malformed


In [7]:
# Create a connection to the `SQLite` file for querying.
sra_dbname <- "F:/SRAmetadb.sqlite"
sracon <- dbConnect(SQLite(),sra_dbname) 
sracon <- dbConnect(dbDriver("SQLite"), sra_dbname)
sracon

<SQLiteConnection>
  Path: F:\SRAmetadb.sqlite
  Extensions: TRUE

In [8]:
# To investigate the content of the database, define the queries that should be run, determine the fields in the data:
sraTables <- dbListTables(sracon)
dbListFields(sracon,"study")

ERROR: Error: no such table: study


In [10]:
sraTables

character(0)

In [ ]:
## frame the first query to get the accessions and titles from the SRA study where,
## the study type contains the keyword `embryo`. 
## The query can be seen as a typical SQL query:
myHit <- dbGetQuery(sracon, 
                    paste("select study_accession,study_title from study where","study_description like'%embryo'", sep=" "))
myHit

In [ ]:
# To do a free text search for the terms of interest
myHit <- getSRA( search_terms = "brain", out_types = c('run','study'), sracon)
head(myHit)

In [ ]:
# Combine key words for appropriate logic, and take a look at some of the retrieved results.
myHit <- getSRA( search_terms ='Alzheimers OR "EPILEPSY"', out_types = c('sample'), sracon)
head(myHit)

## 2. Downloading data from the SRA database

After querying the `SRA` database, we can use the `SRAdb` package to download the `FASTQ` data based on the queries.
Before that, we will need the queries we want to search for or the ID of the data we want to retrieve.

In [ ]:
# load the SRAdb library at first.
library(SRAdb)

In [ ]:
## Frame a query. For instance, use the same query as in the previous recipe to search for `ALZHEIMERS` or `EPILEPSY`，
## note that the step requires the `sracon` object created in the previous recipe.
sra_dbname <- 'F:/SRAmetadb.sqlite'
sracon <- dbConnect(SQLite(),sra_dbname) # sracon <- dbConnect(dbDriver("SQLite"), sra_dbname)
sracon
myHit <- getSRA( search_terms ='ALZHEIMERS OR "EPILEPSY"', out_types = c('sample'), sracon)
head(myHit)

In [ ]:
## Choose two of the hits and find their accessions (IDs) using the `sraConvert` function. 
## And take a look at the object created
conversion <- sraConvert( c('ERS354366','SRS266589'), sra_con = sracon)
conversion

In [ ]:
## To get information on one of the experiments, use the `getSRAinfo` function
rs <- getSRAinfo( c("SRX100465"), sracon, sraType = "sra")
rs

In [ ]:
## Use the `getSRAfile` function to download the data of interest (the run). 
## Due to some problems, the `download.file()` can't work, so I download it on the internet via the `ftp site`
getSRAfile( c("SRR351672", "SRR351673"), sracon, fileType = 'fastq')  

This part deviates in the terms of the conversion of the IDs from the results of the query for free text search. This gives us the accession IDs for the hits retrieved from SRA.<br>
The `getSRAfile` function fetches the file from the SRA database.

## 3. Reading FASTQ files in R

A `FASTQ` file, can have several short sequence reads in it. It is the universally accepted format in the NGS community and for most alignment programs such as Bowtie, which uses these files as input. In order to analyze the data, we need to read in this data in the workspace. <br>
We can download a file from the SRA database, or use the example file in the `ShortRead` package.

In [ ]:
# download files from within R
install.packages("R.utils")
library(R.utils)
download.file(url="ftp://ftp.ddbj.nig.ac.jp/ddbj_database/dra/fastq/SRA146/SRA146985/SRX495419/SRR1198924.fastq.bz2", 
              destfile = "F:/SRR1198924.fastq.bz2")
# It is a `.bz` file, use "bunzip()" to decompress
# download.file(url="ftp://ftp.ddbj.nig.ac.jp/ddbj_database/dra/fastq/SRA000/SRA000241/SRA000241.run.xml", 
#              destfile = "D:/Try-practice/Chapter-8/3-read fastq files/SRA000241.xml")    
# As before there are `fastq` files, but now they are `xml` files.

In [ ]:
# unzip the downloaded file.
bunzip2(file = "F:/SRR1198924.fastq.bz2")

In [ ]:
download.file(url="ftp://ftp.ddbj.nig.ac.jp/ddbj_database/dra/fastq/SRA000/SRA000241/SRA000241.run.xml", 
             destfile = "F:/SRA000241.xml")  

In [ ]:
library(ShortRead)  # load the ShortRead package

To read the downloaded FASTQ file as an R object, use the `readFastq` function. Note that `FASTQ` files are usually heavy and might require huge memory space while reading (it might need approximately 8 GB); therefore, check the machines before working on such files.

In [ ]:
MyFastq <- readFastq("F:/", pattern=".fastq")
MyFastq

In [ ]:
# Look at the first four lines of the file by the generic `readLines` function
readLines("F:/SRR1198924.fastq", 4)

In [ ]:
library(XML) 
xmlread <- xmlTreeParse(readLines("F:/SRA000241.xml"), asText=TRUE) # readLines()????????
xmlread

In [ ]:
xmltop <- xmlRoot(xmlread) # Get the top-level XML node.#形成根目录列表数据
xmltop

In [ ]:
class(xmltop) # 对象中的类
xmlName(xmltop) #第一层（根目录）节点的名称
xmlSize(xmltop) #根目录大小，多少
xmlName(xmltop[[1]]) # 查看子目录名

In [ ]:
# 匹配查询序列信息
getNodeSet(xmltop, "/RUN_SET//PRIMARY_ID")   # subset by primary ID
getNodeSet(xmltop, "/RUN_SET//SUBMITTER_ID")
getNodeSet(xmltop, "/RUN_SET//RUN_ATTRIBUTE")
srr_648 <- getNodeSet(xmltop, "/RUN_SET//RUN[@accession = 'SRR000648']") # srr_648 <- xmltop[2]
srr_648 <- xmlToDataFrame(srr_648)
names(srr_648) <- NULL
class(srr_648)
srr_648 <- as.character(unlist(srr_648[4]))
class(srr_648)
srr_648 <- strsplit(srr_648, split = "sequence")
list(srr_648)
srr_648 <- srr_648[[1]]
srr_648[2]
srr_648 <- strsplit(srr_648[2], split="key")
class(648)
srr_648 <- unlist(srr_648)
srr_648[1]
srr_648 <- as.character(srr_648[1])
srr_648

In [ ]:
srr_657 <- getNodeSet(xmltop, "/RUN_SET//RUN[@accession = 'SRR000657']") # srr_657 <- xmltop[11]
srr_657 <- xmlToDataFrame(srr_657)
names(srr_657) <- NULL
class(srr_657)
srr_657 <- as.character(unlist(srr_657[4]))
class(srr_657)
srr_657 <- strsplit(srr_657, split = "sequence")
list(srr_657)
srr_657 <- srr_657[[1]]
srr_657[2]
srr_657 <- strsplit(srr_657[2], split="key")
class(srr_657)
srr_657 <- unlist(srr_657)
srr_657[1]
srr_657 <- as.character(srr_657[1])
srr_657
srr_648

In [ ]:
example <- rbind(srr_648, srr_657)
example
class(example)